In [11]:
pip install -U webdriver-manager selenium


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


EXTRAER URLS DE datos_pontevedra_Municipio_limpio , no lo recomiendo por que la estructura para cada tipo como restaurante ,que hacer ,hoteles cambian su desarrollo 

In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import urllib.parse
import re
import random

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
# options.add_argument("--headless")  # Descomenta si quieres ejecutar sin abrir la ventana

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# === 2. Leer CSV existente ===
csv_file = r"datos_pontevedra_Municipio_limpio.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig", sep=';')

# Añadir columnas si no existen
for col in [
    "nombre_tripadvisor",
    "link_tripadvisor",
    "calificacion_tripadvisor",
    "ubicacion_tripadvisor",
    "web_tripadvisor",
    "telefono_tripadvisor",
    "correo_tripadvisor"
]:
    if col not in df.columns:
        df[col] = None

# === 3. Progreso previo ===
progreso_file = "progreso_tripadvisor_nombre.txt"
if os.path.exists(progreso_file):
    with open(progreso_file, "r") as f:
        last_index = int(f.read().strip())
    print(f"♻️ Retomando desde la fila {last_index}")
else:
    last_index = 0


# === Funciones auxiliares ===
def parse_numeric(s):
    if not s:
        return None
    s = s.replace(",", ".")
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    return m.group(1) if m else None


def extract_rating(driver):
    xpaths_try = [
        "//div[contains(@class,'VVbkp')]//span[@aria-hidden='true']",
        "//div[contains(@class,'MyMKp') or contains(@class,'MyMKp u')]//span[@aria-hidden='true']",
        "//span[contains(@aria-label,'of 5') or contains(@aria-label,'de 5')]",
        "//span[contains(@class,'bubble_')]",
        "//img[contains(@alt,'of 5') or contains(@alt,'de 5')]",
        "//h1//following::span[@aria-hidden='true'][1]",
        "//h1//following::span[contains(text(),'.') or contains(text(),',')][1]",
    ]

    for xp in xpaths_try:
        elems = driver.find_elements(By.XPATH, xp)
        if not elems:
            continue
        for e in elems:
            text = (e.get_attribute("textContent") or e.text or "").strip()
            if text:
                val = parse_numeric(text)
                if val:
                    return val
            aria = (e.get_attribute("aria-label") or "").strip()
            if aria:
                val = parse_numeric(aria)
                if val:
                    return val
            alt = (e.get_attribute("alt") or "").strip()
            if alt:
                val = parse_numeric(alt)
                if val:
                    return val
            cls = (e.get_attribute("class") or "")
            m = re.search(r"bubble_(\d+)", cls)
            if m:
                return str(int(m.group(1)) / 10)
    return None


def extract_contact_info(driver):
    """Extrae web, teléfono y correo desde el bloque de contactos."""
    web, tel, correo = None, None, None
    try:
        links = driver.find_elements(
            By.XPATH,
            "//div[contains(@class,'CwizA')]/a[contains(@class,'xyffN')]"
        )
        for link in links:
            href = link.get_attribute("href") or ""
            if href.startswith("tel:"):
                tel = href.replace("tel:", "").strip()
            elif href.startswith("mailto:"):
                correo = href.replace("mailto:", "").strip()
            elif href.startswith("http"):
                web = href.strip()
    except Exception:
        pass
    return web, tel, correo


def extract_location(driver):
    """
    Extrae la ubicación de la página usando las clases y estructuras específicas proporcionadas.
    """
    posibles_xpaths = [
        # Buscar ubicación dentro de los divs sin span
        "//div[contains(@class,'vcYDc P e H o')]//div[contains(@class,'biGQs _P ezezH')]",
        # Otra opción de búsqueda para la dirección
        "//*[contains(@data-automation,'restaurantsMapLinkOnName')]//span",  # Ya tenías esta búsqueda
    ]

    for xp in posibles_xpaths:
        elems = driver.find_elements(By.XPATH, xp)
        for e in elems:
            texto = (e.text or "").strip()
            if texto and len(texto) > 5:
                # Filtro de palabras clave que no son direcciones
                if not any(bad in texto.lower() for bad in [
                    "opinión", "reseña", "comentario", "ver más", "página web", "reviews", "review"
                ]):
                    print(f"[DEBUG] Dirección encontrada: {texto}")
                    return texto

    return None

def extract_about(driver):
    """
    Extrae el texto del "Acerca de" desde la clase 'NbcSk e'.
    """
    try:
        about_elem = driver.find_element(By.XPATH, "//div[contains(@class,'NbcSk e')]")
        return about_elem.text.strip() if about_elem else None
    except Exception as e:
        print(f"⚠️ Error extrayendo el 'Acerca de': {e}")
        return None

def extract_features(driver):
    """
    Extrae el texto antes y después de la palabra "Features" en la página.
    """
    try:
        # Buscar el texto completo de la página o de la sección que contiene "Features"
        full_text = driver.find_element(By.XPATH, "//div[contains(@class,'NbcSk e')]").text

        # Buscar la palabra "Features"
        if "Features" in full_text:
            # Dividir el texto antes y después de "Features"
            before_features, after_features = full_text.split("Features", 1)
            return before_features.strip(), after_features.strip()
        else:
            return None, None
    except Exception as e:
        print(f"⚠️ Error extrayendo las características: {e}")
        return None, None

def extract_hours(driver):
    """
    Extrae los horarios generales y los horarios por día desde las clases 'f e' y 'f e Q3'.
    """
    general_hours = None
    daily_hours = None

    try:
        # Extraer horarios generales
        general_elem = driver.find_elements(By.XPATH, "//div[contains(@class,'f e')]")
        if general_elem:
            general_hours = [elem.text.strip() for elem in general_elem]
        
        # Extraer horarios por día
        daily_elem = driver.find_elements(By.XPATH, "//div[contains(@class,'f e Q3')]")
        if daily_elem:
            daily_hours = [elem.text.strip() for elem in daily_elem]
        
        return general_hours, daily_hours
    except Exception as e:
        print(f"⚠️ Error extrayendo horarios: {e}")
        return None, None


def scroll_to_bottom(driver, max_scrolls=20, wait_time=2):
    """Desplaza la página hacia abajo repetidamente para cargar todo el contenido dinámico."""
    last_height = driver.execute_script("return document.body.scrollHeight")  # Obtén la altura inicial

    for _ in range(max_scrolls):
        # Desplázate hasta el fondo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)  # Espera para que el contenido cargue

        # Compara la altura actual con la anterior
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Si la altura no cambia, significa que ya hemos llegado al final
        if new_height == last_height:
            break
        
        last_height = new_height  # Actualiza la altura para el siguiente desplazamiento
  
        
# === 4. Recorrer filas ===
for i, row in df.iloc[last_index:].iterrows():
    nombre = str(row["nombre"]).strip() if "nombre" in df.columns else None
    if not nombre or pd.isna(nombre):
        continue

    print(f"\n🌍 Buscando en TripAdvisor: {nombre}")

    try:
        ubicacion = "Pontevedra Galicia España"
        query = urllib.parse.quote(f"{nombre} {ubicacion}")
        search_url = f"https://www.tripadvisor.com/Search?q={query}"
        driver.get(search_url)
        time.sleep(random.uniform(8, 12))  # búsqueda más rápida

        # Buscar primer resultado válido
        try:
            primer_link = wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, "(//a[contains(@href,'/Hotel_') or contains(@href,'/Attraction_') or contains(@href,'/Restaurant_')])[1]")
                )
            )
            href = primer_link.get_attribute("href")
        except:
            print(f"⚠️ No se encontró resultado válido para '{nombre}'")
            df.at[i, "nombre_tripadvisor"] = "No encontrado"
            df.to_csv("Datos_Turismo_tripadvisor_nombres.csv", index=False, encoding="utf-8-sig", sep=';')
            with open(progreso_file, "w") as f:
                f.write(str(i + 1))
            continue

        # Entrar al resultado
        driver.get(href)
        time.sleep(random.uniform(10, 20))  # espera más corta, máximo 20s dentro del perfil

        # Desplazar hacia abajo para cargar contenido dinámico
        scroll_to_bottom(driver, max_scrolls=30, wait_time=3)  # Desplazamiento con tiempo de espera

        # Extraer nombre principal
        try:
            nombre_elem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ReGgO")))
            nombre_tripadvisor = nombre_elem.text.strip()
        except:
            nombre_tripadvisor = None

        # Extraer calificación
        calificacion = extract_rating(driver)

        # Extraer ubicación mejorada
        ubicacion_texto = extract_location(driver)

        # Extraer contacto (web, teléfono, correo)
        web, tel, correo = extract_contact_info(driver)

        # Extraer 'Acerca de' desde la clase NbcSk e
        acerca_de = extract_about(driver)

        # Extraer horarios generales y horarios por día
        general_hours, daily_hours = extract_hours(driver)

        # Guardar resultados
        df.at[i, "nombre_tripadvisor"] = nombre_tripadvisor
        df.at[i, "link_tripadvisor"] = href
        df.at[i, "calificacion_tripadvisor"] = calificacion
        df.at[i, "ubicacion_tripadvisor"] = ubicacion_texto
        df.at[i, "web_tripadvisor"] = web
        df.at[i, "telefono_tripadvisor"] = tel
        df.at[i, "correo_tripadvisor"] = correo
        df.at[i, "acerca_de_tripadvisor"] = acerca_de  # Añadir esta columna
        df.at[i, "general_hours"] = ", ".join(general_hours) if general_hours else None  # Horarios generales
        df.at[i, "daily_hours"] = ", ".join(daily_hours) if daily_hours else None    # Horarios por día

        print(f"✅ Nombre: {nombre_tripadvisor if nombre_tripadvisor else 'No encontrado'} | ⭐ {calificacion if calificacion else 'No disponible'}")
        print(f"📍 {ubicacion_texto if ubicacion_texto else 'Sin dirección'} | 🌐 {web if web else 'Sin web'} | 📞 {tel if tel else 'Sin teléfono'} | ✉️ {correo if correo else 'Sin correo'}")
        print(f"📝 Acerca de: {acerca_de if acerca_de else 'No disponible'}")
        print(f"🕒 Horarios generales: {general_hours if general_hours else 'No disponibles'}")
        print(f"🗓️ Horarios por día: {daily_hours if daily_hours else 'No disponibles'}")

        # Guardar CSV y progreso
        df.to_csv("Datos_Turismo_tripadvisor_nombres.csv", index=False, encoding="utf-8-sig", sep=';')
        with open(progreso_file, "w") as f:
            f.write(str(i + 1))

        # Espera entre búsquedas (máximo 40s total)
        pausa = random.uniform(10, 25)
        print(f"🕒 Esperando {pausa:.1f} segundos antes de la siguiente búsqueda...")
        time.sleep(pausa)

        if (i + 1) % 10 == 0:
            descanso = random.uniform(40, 60)
            print(f"😴 Descanso largo de {descanso:.0f} segundos para evitar bloqueos...")
            time.sleep(descanso)

    except Exception as e:
        print(f"⚠️ Error procesando '{nombre}': {e}")
        with open(progreso_file, "w") as f:
            f.write(str(i))
        continue

driver.quit()
print("\n✅ Proceso completado. CSV actualizado y progreso guardado.")


🌍 Extrayendo URLs de la página 1...
🌍 Extrayendo URLs de la página 2...
🌍 Extrayendo URLs de la página 3...
🌍 Extrayendo URLs de la página 4...
🌍 Extrayendo URLs de la página 5...

✅ Proceso completado. CSV con URLs de hoteles guardado.


EXTRAER URLS DE restaurantes =solo urls para despues extraer el resto

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import pandas as pd

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
# options.add_argument("--headless")  # Descomenta si quieres ejecutar sin abrir la ventana

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)  # Aumentamos el tiempo de espera para WebDriverWait

# === 2. Inicialización del DataFrame para las URLs ===
hotel_urls = []  # Lista para almacenar las URLs de los hoteles

# === Función para extraer las URLs de los hoteles ===
def extract_hotel_urls(driver):
    """Extrae las URLs de los hoteles de la página actual"""
    urls = []
    try:
        # Encuentra todos los elementos con la clase que contiene las URLs de los hoteles
        hotel_elements = driver.find_elements(By.XPATH, "//a[contains(@class, 'BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS')]")
        for element in hotel_elements:
            href = element.get_attribute("href")
            if href:
                urls.append(href)
    except Exception as e:
        print(f"⚠️ Error extrayendo URLs de hoteles: {e}")
    return urls

# === Función para ir a la siguiente página usando el offset ===
def go_to_next_page_by_offset(driver, offset):
    """Navega a la siguiente página modificando el offset en la URL"""
    try:
        # Actualiza el offset para ir a la siguiente página
        next_url = f"https://www.tripadvisor.com/Hotels-g488307-oa{offset}-zfd21371-a_ufe.true-Pontevedra_Province_of_Pontevedra_Galicia-Hotels.html"
        driver.get(next_url)  # Navega a la nueva URL con el nuevo offset
        time.sleep(random.uniform(6, 12))  # Espera para que la página cargue
        return True
    except Exception as e:
        print(f"⚠️ Error al navegar a la siguiente página con offset {offset}: {e}")
        return False

# === Recorrer las páginas ===
url_base = "https://www.tripadvisor.com/Hotels-g488307-oa0-zfd21371-a_ufe.true-Pontevedra_Province_of_Pontevedra_Galicia-Hotels.html"
driver.get(url_base)
time.sleep(random.uniform(30, 40))  # Espera para que la página cargue

# Extraer las URLs de la primera página
hotel_urls.extend(extract_hotel_urls(driver))

# Límite manual de páginas
max_pages = 5  # Establecer el número máximo de páginas a recorrer
page_number = 1
offset = 30  # El offset de la segunda página es 30, y así sucesivamente

while page_number <= max_pages:
    print(f"🌍 Extrayendo URLs de la página {page_number}...")
    page_number += 1
    
    # Navegar a la siguiente página usando el offset
    if not go_to_next_page_by_offset(driver, offset):
        break  # Si no hay más páginas, salimos del bucle
    
    # Extraer URLs de la nueva página
    hotel_urls.extend(extract_hotel_urls(driver))

    # Incrementar el offset para la siguiente página (cada página tiene 30 resultados)
    offset += 30

# === Guardar las URLs en un DataFrame ===
df_urls = pd.DataFrame(hotel_urls, columns=["hotel_url"])

# Guardar las URLs en un archivo CSV
df_urls.to_csv("hotel_urls_pontevedra.csv", index=False, encoding="utf-8-sig", sep=';')

print("\n✅ Proceso completado. CSV con URLs de hoteles guardado.")


🌍 Extrayendo URLs de la página 1...
🌍 Extrayendo URLs de la página 2...
🌍 Extrayendo URLs de la página 3...
🌍 Extrayendo URLs de la página 4...
🌍 Extrayendo URLs de la página 5...

✅ Proceso completado. CSV con URLs de hoteles guardado.


extraer urls de hoteles , revisar creo que son iguales las dos pero se pueden guiar para terminar el desarrollo 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import pandas as pd

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
# options.add_argument("--headless")  # Descomenta si quieres ejecutar sin abrir la ventana

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)  # Aumentamos el tiempo de espera para WebDriverWait

# === 2. Inicialización del DataFrame para las URLs ===
hotel_urls = []  # Lista para almacenar las URLs de los hoteles

# === Función para extraer las URLs de los hoteles ===
def extract_hotel_urls(driver):
    """Extrae las URLs de los hoteles de la página actual"""
    urls = []
    try:
        # Encuentra todos los elementos con la clase que contiene las URLs de los hoteles
        hotel_elements = driver.find_elements(By.XPATH, "//a[contains(@class, 'BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS')]")
        for element in hotel_elements:
            href = element.get_attribute("href")
            if href:
                urls.append(href)
    except Exception as e:
        print(f"⚠️ Error extrayendo URLs de hoteles: {e}")
    return urls

# === Función para ir a la siguiente página usando el offset ===
def go_to_next_page_by_offset(driver, offset):
    """Navega a la siguiente página modificando el offset en la URL"""
    try:
        # Actualiza el offset para ir a la siguiente página
        next_url = f"https://www.tripadvisor.com/Hotels-g488307-oa{offset}-zfd21371-a_ufe.true-Pontevedra_Province_of_Pontevedra_Galicia-Hotels.html"
        driver.get(next_url)  # Navega a la nueva URL con el nuevo offset
        time.sleep(random.uniform(15, 20))  # Espera más larga para asegurar carga completa de la página
        return True
    except Exception as e:
        print(f"⚠️ Error al navegar a la siguiente página con offset {offset}: {e}")
        return False

# === Recorrer las páginas ===
url_base = "https://www.tripadvisor.com/Hotels-g488307-oa0-zfd21371-a_ufe.true-Pontevedra_Province_of_Pontevedra_Galicia-Hotels.html"
driver.get(url_base)
time.sleep(random.uniform(30, 40))  # Espera más larga para asegurar carga completa de la página

# Extraer las URLs de la primera página
hotel_urls.extend(extract_hotel_urls(driver))

# Límite manual de páginas
max_pages = 3  # Establecer el número máximo de páginas a recorrer
page_number = 1
offset = 30  # El offset de la segunda página es 30, y así sucesivamente

while page_number <= max_pages:
    print(f"🌍 Extrayendo URLs de la página {page_number}...")
    page_number += 1
    
    # Navegar a la siguiente página usando el offset
    if not go_to_next_page_by_offset(driver, offset):
        break  # Si no hay más páginas, salimos del bucle
    
    # Extraer URLs de la nueva página
    hotel_urls.extend(extract_hotel_urls(driver))

    # Incrementar el offset para la siguiente página (cada página tiene 30 resultados)
    offset += 30

# === Guardar las URLs en un DataFrame ===
df_urls = pd.DataFrame(hotel_urls, columns=["hotel_url"])

# Guardar las URLs en un archivo CSV
df_urls.to_csv("hotel_urls_pontevedra.csv", index=False, encoding="utf-8-sig", sep=';')

print("\n✅ Proceso completado. CSV con URLs de hoteles guardado.")


🌍 Extrayendo URLs de la página 1...
🌍 Extrayendo URLs de la página 2...
🌍 Extrayendo URLs de la página 3...

✅ Proceso completado. CSV con URLs de hoteles guardado.


leer actividades por hacer  para extraer solo urls

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import pandas as pd

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
# options.add_argument("--headless")  # Descomenta si quieres ejecutar sin abrir la ventana

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)  # Aumentamos el tiempo de espera para WebDriverWait

# === 2. Inicialización del DataFrame para las URLs ===
activity_urls = []  # Lista para almacenar las URLs de las actividades

# === Función para extraer las URLs de las actividades ===
def extract_activity_urls(driver):
    """Extrae las URLs de las actividades de la página actual y filtra las URLs con 'ClientLink'"""
    urls = []
    try:
        # Encuentra todos los enlaces <a> que contienen exactamente la clase "BMQDV _F Gv wSSLS SwZTJ"
        activity_elements = driver.find_elements(By.XPATH, "//a[@class='BMQDV _F Gv wSSLS SwZTJ']")
        
        # Asegurarse de que estamos extrayendo solo los enlaces correctos
        for element in activity_elements:
            href = element.get_attribute("href")
            if href and "ClientLink" not in href:  # Filtrar los enlaces que contienen "ClientLink"
                urls.append(href)
    except Exception as e:
        print(f"⚠️ Error extrayendo URLs de actividades: {e}")
    return urls

# === Función para ir a la siguiente página usando el offset ===
def go_to_next_page_by_offset(driver, offset):
    """Navega a la siguiente página modificando el offset en la URL"""
    try:
        # Actualiza el offset para ir a la siguiente página
        next_url = f"https://www.tripadvisor.com/Attractions-g488307-Activities-oa{offset}-Pontevedra_Province_of_Pontevedra_Galicia.html"
        driver.get(next_url)  # Navega a la nueva URL con el nuevo offset
        time.sleep(random.uniform(30, 40))  # Espera más larga para asegurar carga completa de la página
        return True
    except Exception as e:
        print(f"⚠️ Error al navegar a la siguiente página con offset {offset}: {e}")
        return False

# === Recorrer las páginas ===
url_base = "https://www.tripadvisor.com/Attractions-g488307-Activities-oa0-Pontevedra_Province_of_Pontevedra_Galicia.html"
driver.get(url_base)
time.sleep(random.uniform(30, 40))  # Espera más larga para asegurar carga completa de la página

# Extraer las URLs de la primera página
activity_urls.extend(extract_activity_urls(driver))

# Límite manual de páginas
max_pages = 4  # Establecer el número máximo de páginas a recorrer
page_number = 1
offset = 30  # El offset de la segunda página es 30, y así sucesivamente

while page_number <= max_pages:
    print(f"🌍 Extrayendo URLs de la página {page_number}...")
    page_number += 1
    
    # Navegar a la siguiente página usando el offset
    if not go_to_next_page_by_offset(driver, offset):
        break  # Si no hay más páginas, salimos del bucle
    
    # Extraer URLs de la nueva página
    activity_urls.extend(extract_activity_urls(driver))

    # Incrementar el offset para la siguiente página (cada página tiene 30 resultados)
    offset += 30

# === Guardar las URLs en un DataFrame ===
df_urls = pd.DataFrame(activity_urls, columns=["activity_url"])

# Guardar las URLs en un archivo CSV
df_urls.to_csv("activity_urls_pontevedra_filtered.csv", index=False, encoding="utf-8-sig", sep=';')

print("\n✅ Proceso completado. CSV con URLs de actividades guardado.")


🌍 Extrayendo URLs de la página 1...
🌍 Extrayendo URLs de la página 2...
🌍 Extrayendo URLs de la página 3...
🌍 Extrayendo URLs de la página 4...

✅ Proceso completado. CSV con URLs de actividades guardado.


leer csv de restaurantes para extraer mas info 

In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import os
import re

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
# options.add_argument("--headless")  # Descomenta si quieres ejecutar sin abrir la ventana

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# === 2. Leer CSV con URLs de restaurantes ===
csv_urls = "datos/restaurant_urls_pontevedra.csv"
df_urls = pd.read_csv(csv_urls, encoding="utf-8-sig", sep=';')

# === 3. Crear CSV de resultados si no existe ===
results_file = "Datos_Restaurantes_Tripadvisor.csv"
if os.path.exists(results_file):
    df_results = pd.read_csv(results_file, encoding="utf-8-sig", sep=';')
else:
    df_results = df_urls.copy()
    df_results["nombre_tripadvisor"] = None
    df_results["calificacion_tripadvisor"] = None
    df_results["ubicacion_tripadvisor"] = None
    df_results["web_tripadvisor"] = None
    df_results["telefono_tripadvisor"] = None
    df_results["correo_tripadvisor"] = None
    df_results["acerca_de_tripadvisor"] = None
    df_results["general_hours"] = None
    df_results["daily_hours"] = None
    df_results["features_before"] = None
    df_results["features_after"] = None

# === 4. Progreso previo ===
progreso_file = "progreso_restaurant_urls.txt"
if os.path.exists(progreso_file):
    with open(progreso_file, "r") as f:
        last_index = int(f.read().strip())
    print(f"♻️ Retomando desde la fila {last_index}")
else:
    last_index = 0

# === Funciones auxiliares exactas de tu código ===

def parse_numeric(s):
    if not s:
        return None
    s = s.replace(",", ".")
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    return m.group(1) if m else None

def extract_rating(driver):
    xpaths_try = [
        "//div[contains(@class,'VVbkp')]//span[@aria-hidden='true']",
        "//div[contains(@class,'MyMKp') or contains(@class,'MyMKp u')]//span[@aria-hidden='true']",
        "//span[contains(@aria-label,'of 5') or contains(@aria-label,'de 5')]",
        "//span[contains(@class,'bubble_')]",
        "//img[contains(@alt,'of 5') or contains(@alt,'de 5')]",
        "//h1//following::span[@aria-hidden='true'][1]",
        "//h1//following::span[contains(text(),'.') or contains(text(),',')][1]",
    ]
    for xp in xpaths_try:
        elems = driver.find_elements(By.XPATH, xp)
        if not elems:
            continue
        for e in elems:
            text = (e.get_attribute("textContent") or e.text or "").strip()
            if text:
                val = parse_numeric(text)
                if val:
                    return val
            aria = (e.get_attribute("aria-label") or "").strip()
            if aria:
                val = parse_numeric(aria)
                if val:
                    return val
            alt = (e.get_attribute("alt") or "").strip()
            if alt:
                val = parse_numeric(alt)
                if val:
                    return val
            cls = (e.get_attribute("class") or "")
            m = re.search(r"bubble_(\d+)", cls)
            if m:
                return str(int(m.group(1)) / 10)
    return None

def extract_contact_info(driver):
    web, tel, correo = None, None, None
    try:
        links = driver.find_elements(By.XPATH, "//div[contains(@class,'CwizA')]/a[contains(@class,'xyffN')]")
        for link in links:
            href = link.get_attribute("href") or ""
            if href.startswith("tel:"):
                tel = href.replace("tel:", "").strip()
            elif href.startswith("mailto:"):
                correo = href.replace("mailto:", "").strip()
            elif href.startswith("http"):
                web = href.strip()
    except:
        pass
    return web, tel, correo

def extract_location(driver):
    posibles_xpaths = [
        "//div[contains(@class,'vcYDc P e H o')]//div[contains(@class,'biGQs _P ezezH')]",
        "//*[contains(@data-automation,'restaurantsMapLinkOnName')]//span",
    ]
    for xp in posibles_xpaths:
        elems = driver.find_elements(By.XPATH, xp)
        for e in elems:
            texto = (e.text or "").strip()
            if texto and len(texto) > 5:
                if not any(bad in texto.lower() for bad in ["opinión","reseña","comentario","ver más","página web","reviews","review"]):
                    return texto
    return None

def extract_about(driver):
    try:
        about_elem = driver.find_element(By.XPATH, "//div[contains(@class,'NbcSk e')]")
        return about_elem.text.strip() if about_elem else None
    except:
        return None

def extract_features(driver):
    try:
        full_text = driver.find_element(By.XPATH, "//div[contains(@class,'NbcSk e')]").text
        if "Features" in full_text:
            before, after = full_text.split("Features",1)
            return before.strip(), after.strip()
        else:
            return None, None
    except:
        return None, None

def extract_hours(driver):
    general_hours, daily_hours = None, None
    try:
        general_elem = driver.find_elements(By.XPATH, "//div[contains(@class,'f e')]")
        if general_elem:
            general_hours = [elem.text.strip() for elem in general_elem]
        daily_elem = driver.find_elements(By.XPATH, "//div[contains(@class,'f e Q3')]")
        if daily_elem:
            daily_hours = [elem.text.strip() for elem in daily_elem]
        return general_hours, daily_hours
    except:
        return None, None

def scroll_to_bottom(driver, max_scrolls=20, wait_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# === 5. Recorrer cada URL y guardar progresivamente ===
for i, row in df_urls.iloc[last_index:].iterrows():
    url = row["restaurant_url"]
    if not url:
        continue

    print(f"\n🌍 Procesando: {url}")
    try:
        driver.get(url)
        time.sleep(random.uniform(20, 30))
        scroll_to_bottom(driver, max_scrolls=30, wait_time=3)

        # Extraer datos exactamente como antes
        nombre = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ReGgO"))).text.strip()
        calificacion = extract_rating(driver)
        ubicacion = extract_location(driver)
        web, tel, correo = extract_contact_info(driver)
        acerca = extract_about(driver)
        before_features, after_features = extract_features(driver)
        #general_hours, daily_hours = extract_hours(driver)

        # Guardar en DataFrame
        df_results.loc[df_results["restaurant_url"] == url, "nombre_tripadvisor"] = nombre
        df_results.loc[df_results["restaurant_url"] == url, "calificacion_tripadvisor"] = calificacion
        df_results.loc[df_results["restaurant_url"] == url, "ubicacion_tripadvisor"] = ubicacion
        df_results.loc[df_results["restaurant_url"] == url, "web_tripadvisor"] = web
        df_results.loc[df_results["restaurant_url"] == url, "telefono_tripadvisor"] = tel
        df_results.loc[df_results["restaurant_url"] == url, "correo_tripadvisor"] = correo
        df_results.loc[df_results["restaurant_url"] == url, "acerca_de_tripadvisor"] = acerca
        df_results.loc[df_results["restaurant_url"] == url, "features_before"] = before_features
        df_results.loc[df_results["restaurant_url"] == url, "features_after"] = after_features
        #df_results.loc[df_results["restaurant_url"] == url, "general_hours"] = ", ".join(general_hours) if general_hours else None
        #df_results.loc[df_results["restaurant_url"] == url, "daily_hours"] = ", ".join(daily_hours) if daily_hours else None

        # Guardar CSV y progreso continuamente
        df_results.to_csv(results_file, index=False, encoding="utf-8-sig", sep=';')
        with open(progreso_file, "w") as f:
            f.write(str(i + 1))

        # Espera entre URLs
        pausa = random.uniform(20, 30)
        print(f"🕒 Esperando {pausa:.1f} segundos antes de la siguiente URL...")
        time.sleep(pausa)

    except Exception as e:
        print(f"⚠️ Error procesando {url}: {e}")
        with open(progreso_file, "w") as f:
            f.write(str(i))
        continue

driver.quit()
print("\n✅ Proceso completado. CSV actualizado y guardado.")


♻️ Retomando desde la fila 0

🌍 Procesando: https://www.tripadvisor.com/Restaurant_Review-g488307-d23942003-Reviews-Gumer_Pontevedra-Pontevedra_Province_of_Pontevedra_Galicia.html
🕒 Esperando 27.7 segundos antes de la siguiente URL...

🌍 Procesando: https://www.tripadvisor.com/Restaurant_Review-g488307-d6213155-Reviews-El_Cafetin_de_Pontevedra-Pontevedra_Province_of_Pontevedra_Galicia.html
🕒 Esperando 28.7 segundos antes de la siguiente URL...

🌍 Procesando: https://www.tripadvisor.com/Restaurant_Review-g488307-d10077844-Reviews-La_Quesera_Cheese_Bar-Pontevedra_Province_of_Pontevedra_Galicia.html
🕒 Esperando 26.0 segundos antes de la siguiente URL...

🌍 Procesando: https://www.tripadvisor.com/Restaurant_Review-g488307-d12208923-Reviews-Asador_O_Rancho-Pontevedra_Province_of_Pontevedra_Galicia.html
🕒 Esperando 24.6 segundos antes de la siguiente URL...

🌍 Procesando: https://www.tripadvisor.com/Restaurant_Review-g488307-d1674727-Reviews-PonteTapas-Pontevedra_Province_of_Pontevedra_Galic

In [2]:
import pandas as pd

# === 1. Ruta del archivo CSV que quieres convertir ===
input_csv = r"D:\fontur_lote-1_2025_ws-ARS\scrapeo-Pontevedra\Pontevedra\tripadvisor\Datos_Restaurantes_Tripadvisor.csv"  # Cambia por el nombre de tu archivo CSV
output_excel = "datos_convertidos.xlsx"  # Nombre del archivo Excel resultante

# === 2. Leer el CSV ===

df = pd.read_csv(input_csv, encoding="utf-8-sig", sep=';')

# === 3. Guardar como Excel ===
df.to_excel(output_excel, index=False, engine='openpyxl')

print(f"✅ Conversión completada: '{output_excel}' creado correctamente.")


✅ Conversión completada: 'datos_convertidos.xlsx' creado correctamente.


In [5]:
rmdir /s /q "%USERPROFILE%\.wdm"


C:\Users\seba0\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32\chromedriver.exe - Access is denied.
